## DCGAN with MNIST

* MNIST DATA를 가지고 Deep Convoltuion Gan을 tf.slim을 이용해서 만들어보자

### Setting modules

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import time

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


import tensorflow as tf

slim = tf.contrib.slim
tf.logging.set_verbosity(tf.logging.INFO)

#tf.set_random_seed(219)
#np.random.seed(219)

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
os.environ["CUDA_VISIBLE_DEVICES"]="0"

C:\Users\home\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Hyper Parameters

In [2]:
# Training Flags (hyperparameter configuration)
train_dir = './DCGAN/train/'
max_epochs = 30
save_epochs = 10
summary_steps = 2500
print_steps = 1000
batch_size = 64
learning_rate_D = 0.0002
learning_rate_G = 0.001
k = 1 # the number of step of learning D before learning G
num_samples = 16

In [40]:
tf.reset_default_graph()

### Import MNIST

In [29]:
# Load training and eval data from tf.keras
(train_data, train_labels), _ = tf.keras.datasets.mnist.load_data()

train_data = train_data
train_data = train_data / 255.
train_labels = np.asarray(train_labels, dtype=np.int32)

### Import img_data

In [10]:
import cv2

In [6]:
import os

In [8]:
path_dir = '/Users/home/Downloads/agar_plate_ey/'
file_list = os.listdir(path_dir)

print(path_dir)

/Users/home/Downloads/agar_plate_ey/


In [16]:
path_dir+file_list[9]

'/Users/home/Downloads/agar_plate_ey/img_106.jpg'

In [41]:
train_data = []
for i in enumerate(file_list):
    img = cv2.imread(path_dir + i[1], 0)
    #img = np.array(img)
    #print(img.shape)
    img = img / 255
    train_data.append(img)
    
train_data = np.asanyarray(train_data)

### Set up dataset with `tf.data`

In [42]:
#for train
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
train_dataset = train_dataset.shuffle(buffer_size = 10000)
train_dataset = train_dataset.repeat(count= max_epochs)
train_dataset = train_dataset.batch(batch_size = batch_size)
print(train_dataset)

<BatchDataset shapes: (?, 28, 28), types: tf.float64>


### Create Model

In [43]:
class DCGAN(object):
  """Deep Convolutional Generative Adversarial Networks
  implementation based on http://arxiv.org/abs/1511.06434
  
  "Unsupervised Representation Learning with
  Deep Convolutional Generative Adversarial Networks"
  Alec Radford, Luke Metz and Soumith Chintala
  """
  
  def __init__(self, mode, train_dataset, test_dataset=None):
    """Basic setup.
    
    Args:
      mode (`string`): "train" or "generate".
      train_dataset (`tf.data.Dataset`): train_dataset.
      test_dataset (`tf.data.Dataset`): test_dataset.
    """
    assert mode in ["train", "generate"]
    self.mode = mode
    
    # hyper-parameters for model
    self.x_dim = 28
    self.z_dim = 100
    self.batch_size = batch_size
    self.num_samples = num_samples
    self.train_dataset = train_dataset
    self.test_dataset = test_dataset
    
    # Global step Tensor.
    self.global_step = None
    
    print('The mode is %s.' % self.mode)
    print('complete initializing model.')
    
    
  def build_random_z_inputs(self):
    """Build a vector random_z in latent space.
    
    Returns:
      self.random_z (`4-rank Tensor` with [batch_size, 1, 1, z_dim]):
          latent vector which size is generally 100 dim.
      self.sample_random_z (`4-rank Tensor` with [num_samples, 1, 1, z_dim]):
          latent vector which size is generally 100 dim.
    """
    # 여기를 채워 넣으세요
    # Setup variable of random vector z
    with tf.variable_scope('random_z'):
      self.random_z = tf.random_uniform(shape=[self.batch_size, 1, 1, self.z_dim],
                                        minval=-1.0, maxval=1.0)
      self.sample_random_z = tf.random_uniform(shape=[self.num_samples, 1, 1, self.z_dim],
                                               minval=-1.0, maxval=1.0)

    return self.random_z, self.sample_random_z
  
  
  def read_MNIST(self, dataset):
    """Read MNIST dataset
    
    Args:
      dataset (`tf.data.Dataset` format): MNIST dataset.
      
    Returns:
      self.mnist (`4-rank Tensor` with [batch, x_dim, x_dim, 1]): MNIST dataset with batch size.
    """
    with tf.variable_scope('mnist'):
      iterator = dataset.make_one_shot_iterator()

      self.mnist = iterator.get_next()
      self.mnist = tf.cast(self.mnist, dtype = tf.float32)
      self.mnist = tf.expand_dims(self.mnist, axis=3)
      
    return self.mnist


  def Generator(self, random_z, is_training=True, reuse=False):
    """Generator setup.
    
    Args:
      random_z (`2-rank Tensor` with [batch_size, z_dim]):
          latent vector which size is generally 100 dim.
      is_training (`bool`): whether training mode or test mode.
      reuse (`bool`): whether variable reuse or not.
      
    Returns:
      generated_data (`4-rank Tensor` with [batch_size, h, w, c])
          generated images from random vector z.
    """
    with tf.variable_scope('Generator', reuse=reuse) as scope:
      batch_norm_params = {'decay': 0.9,
                           'epsilon': 0.001,
                           'is_training': is_training,
                           'scope': 'batch_norm'}
      with slim.arg_scope([slim.conv2d_transpose],
                          kernel_size=[4, 4],
                          stride=[2, 2],
                          normalizer_fn=slim.batch_norm,
                          normalizer_params=batch_norm_params):
        # 여기를 채워 넣으세요
        # Use full conv2d_transpose instead of projection and reshape
        # random_z: 1 x 1 x 100 dim
        self.inputs = random_z
        # inputs = random_z: 1 x 1 x 100 dim
        # outputs: 3 x 3 x 256 dim
        self.layer1 = slim.conv2d_transpose(inputs=self.inputs,
                                            num_outputs=256,
                                            kernel_size=[3, 3],
                                            padding='VALID',
                                            scope='layer1')
        # inputs: 3 x 3 x 256 dim
        # outputs: 7 x 7 x 128 dim
        self.layer2 = slim.conv2d_transpose(inputs=self.layer1,
                                            num_outputs=128,
                                            kernel_size=[3, 3],
                                            padding='VALID',
                                            scope='layer2')
        # inputs: 7 x 7 x 128 dim
        # outputs: 14 x 14 x 64 dim
        self.layer3 = slim.conv2d_transpose(inputs=self.layer2,
                                            num_outputs=64,
                                            scope='layer3')
        # inputs: 14 x 14 x 64 dim
        # outputs: 28 x 28 x 1 dim
        self.layer4 = slim.conv2d_transpose(inputs=self.layer3,
                                            num_outputs=1,
                                            normalizer_fn=None,
                                            activation_fn=tf.sigmoid,
                                            scope='layer4')
        # generated_data = outputs: 28 x 28 x 1 dim
        generated_data = self.layer4

        return generated_data
    
    
  def Discriminator(self, data, reuse=False):
    """Discriminator setup.
    
    Args:
      data (`2-rank Tensor` with [batch_size, x_dim]): MNIST real data.
      reuse (`bool`): whether variable reuse or not.
      
    Returns:
      logits (`1-rank Tensor` with [batch_size]): logits of data.
    """
    with tf.variable_scope('Discriminator', reuse=reuse) as scope:
      batch_norm_params = {'decay': 0.9,
                           'epsilon': 0.001,
                           'scope': 'batch_norm'}
      with slim.arg_scope([slim.conv2d],
                          kernel_size=[4, 4],
                          stride=[2, 2],
                          activation_fn=tf.nn.leaky_relu,
                          normalizer_fn=slim.batch_norm,
                          normalizer_params=batch_norm_params):
        # 여기를 채워 넣으세요
        # inputs = data: 28 x 28 x 1 dim
        # outputs: 14 x 14 x 64 dim
        self.layer1 = slim.conv2d(inputs=data,
                                  num_outputs=64,
                                  normalizer_fn=None,
                                  scope='layer1')
        # inputs: 14 x 14 x 64 dim
        # outputs: 7 x 7 x 128 dim
        self.layer2 = slim.conv2d(inputs=self.layer1,
                                  num_outputs=128,
                                  scope='layer2')
        # inputs: 7 x 7 x 128 dim
        # outputs: 3 x 3 x 256 dim
        self.layer3 = slim.conv2d(inputs=self.layer2,
                                  num_outputs=256,
                                  kernel_size=[3, 3],
                                  padding='VALID',
                                  scope='layer3')
        # inputs: 3 x 3 x 256 dim
        # outputs: 1 x 1 x 1 dim
        self.layer4 = slim.conv2d(inputs=self.layer3,
                                  num_outputs=1,
                                  kernel_size=[3, 3],
                                  stride=[1, 1],
                                  padding='VALID',
                                  normalizer_fn=None,
                                  activation_fn=None,
                                  scope='layer4')
        # logits = layer4: 1 x 1 x 1 dim -> 1 dim
        discriminator_logits = tf.squeeze(self.layer4, axis=[1, 2])

        return discriminator_logits
    
    
  def setup_global_step(self):
    """Sets up the global step Tensor."""
    if self.mode == "train":
      self.global_step = tf.train.get_or_create_global_step()
      
      print('complete setup global_step.')
      
      
  def GANLoss(self, logits, is_real=True, scope=None):
    """Computes standard GAN loss between `logits` and `labels`.
    
    Args:
      logits (`1-rank Tensor`): logits.
      is_real (`bool`): True means `1` labeling, False means `0` labeling.
      
    Returns:
      loss (`0-randk Tensor): the standard GAN loss value. (binary_cross_entropy)
    """
    if is_real:
      labels = tf.ones_like(logits)
    else:
      labels = tf.zeros_like(logits)

    # 여기를 채워 넣으세요
    loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=labels,
                                           logits=logits,
                                           scope=scope)

    return loss

      
  def build(self):
    """Creates all ops for training or generate."""
    self.setup_global_step()
    
    if self.mode == "generate":
      pass
    
    else:
      # generating random vector
      self.random_z, self.sample_random_z = self.build_random_z_inputs()
      # read dataset
      self.real_data = self.read_MNIST(self.train_dataset)
      
      # 여기를 채워 넣으세요
      # generating images from Generator() via random vector z
      self.generated_data = self.Generator(self.random_z)
      
      # 여기를 채워 넣으세요
      # discriminating real data by Discriminator()
      self.real_logits = self.Discriminator(self.real_data)
      # discriminating fake data (generated)_images) by Discriminator()
      self.fake_logits = self.Discriminator(self.generated_data, reuse=True)
      
      # 여기를 채워 넣으세요
      # losses of real with label "1"
      self.loss_real = self.GANLoss(self.real_logits, is_real=True, scope='loss_D_real')
      # losses of fake with label "0"
      self.loss_fake = self.GANLoss(self.fake_logits, is_real=False, scope='loss_D_fake')
      
      # losses of Discriminator
      with tf.variable_scope('loss_D'):
        self.loss_Discriminator = self.loss_real + self.loss_fake
        
      # 여기를 채워 넣으세요
      # losses of Generator with label "1" that used to fool the Discriminator
      self.loss_Generator = self.GANLoss(self.fake_logits, is_real=True, scope='loss_G')
      
      # 여기를 채워 넣으세요
      # Separate variables for each function
      self.D_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Discriminator')
      self.G_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Generator')
      
      
      # generating images for sample
      self.sample_data = self.Generator(self.sample_random_z, is_training=False, reuse=True)
      
      # write summaries
      # Add loss summaries
      tf.summary.scalar('losses/loss_Discriminator', self.loss_Discriminator)
      tf.summary.scalar('losses/loss_Generator', self.loss_Generator)
      
      # Add histogram summaries
      for var in self.D_vars:
        tf.summary.histogram(var.op.name, var)
      for var in self.G_vars:
        tf.summary.histogram(var.op.name, var)
      
      # Add image summaries
      tf.summary.image('random_images', self.generated_data, max_outputs=4)
      tf.summary.image('real_images', self.real_data)
      
    print('complete model build.\n')

### Define plot fucntion

In [44]:
def print_sample_data(sample_data, max_print=num_samples):
  print_images = sample_data[:max_print,:]
  print_images = print_images.reshape([max_print, 28, 28])
  print_images = print_images.swapaxes(0, 1)
  print_images = print_images.reshape([28, max_print * 28])
  
  plt.figure(figsize=(max_print, 1))
  plt.axis('off')
  plt.imshow(print_images, cmap='gray')
  plt.show()

## Bulid the model

In [45]:
model = DCGAN(mode="train", train_dataset=train_dataset)
model.build()

# show info for trainable variables
t_vars = tf.trainable_variables()
slim.model_analyzer.analyze_vars(t_vars, print_info=True)

The mode is train.
complete initializing model.
complete setup global_step.
complete model build.

---------
Variables: name (type shape) [size]
---------
Generator/layer1/weights:0 (float32_ref 3x3x256x100) [230400, bytes: 921600]
Generator/layer1/batch_norm/beta:0 (float32_ref 256) [256, bytes: 1024]
Generator/layer2/weights:0 (float32_ref 3x3x128x256) [294912, bytes: 1179648]
Generator/layer2/batch_norm/beta:0 (float32_ref 128) [128, bytes: 512]
Generator/layer3/weights:0 (float32_ref 4x4x64x128) [131072, bytes: 524288]
Generator/layer3/batch_norm/beta:0 (float32_ref 64) [64, bytes: 256]
Generator/layer4/weights:0 (float32_ref 4x4x1x64) [1024, bytes: 4096]
Generator/layer4/biases:0 (float32_ref 1) [1, bytes: 4]
Discriminator/layer1/weights:0 (float32_ref 4x4x1x64) [1024, bytes: 4096]
Discriminator/layer1/biases:0 (float32_ref 64) [64, bytes: 256]
Discriminator/layer2/weights:0 (float32_ref 4x4x64x128) [131072, bytes: 524288]
Discriminator/layer2/batch_norm/beta:0 (float32_ref 128) [

(1087618, 4350472)

In [ ]:
?slim.conv2d_transpos

In [47]:
def setup_global_step(self):
    """Sets up the global step Tensor."""
    if self.mode == "train":
      self.global_step = tf.train.get_or_create_global_step() #전체 training step을 관리하기 위한 함수
      
      print('complete setup global_step.')
      
      
    def GANLoss(self, logits, is_real=True, scope=None):
        """Computes standard GAN loss between `logits` and `labels`.

        Args:
          logits (`1-rank Tensor`): logits.
          is_real (`bool`): True means `1` labeling, False means `0` labeling.

        Returns:
          loss (`0-randk Tensor): the standard GAN loss value. (binary_cross_entropy)
        """
        if is_real:
          labels = tf.ones_like(logits)
        else:
          labels = tf.zeros_like(logits)

        loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=labels, logits=logits, scope=scope)
        return loss

      
    def build(self):
        """Creates all ops for training or generate."""
        self.setup_global_step()

        if self.mode == "generate":
          pass

        else:
          # generating random vector
          self.random_z, self.sample_random_z = self.build_random_z_inputs()
          # read dataset
          self.real_data = self.read_MNIST(self.train_dataset)

          # 여기를 채워 넣으세요
          # generating images from Generator() via random vector z
          self.generated_data = self.Generator(self.random_z, reuse = False)

          # 여기를 채워 넣으세요
          # discriminating real data by Discriminator()
          self.real_logits = self.Discriminator(self.real_data, reuse = False)
          # discriminating fake data (generated)_images) by Discriminator()
          self.fake_logits = self.Discriminator(self.generated_data, reuse = True)

          # 여기를 채워 넣으세요, def GANLoss(self, logits, is_real=True, scope=None)
          # losses of real with label "1"
          self.loss_real = self.GANLoss(self.real_logits, is_real= True, scope='loss_D_real')
          # losses of fake with label "0"
          self.loss_fake = self.GANLoss(self.fake_logits, is_real= False, scope='loss_D_fake')

          # losses of Discriminator
          with tf.variable_scope('loss_D'):
            self.loss_Discriminator = self.loss_real + self.loss_fake

          # 여기를 채워 넣으세요
          # losses of Generator with label "1" that used to fool the Discriminator
          self.loss_Generator = self.GANLoss(self.fake_logits, is_real = True, scope='loss_G')

          # 여기를 채워 넣으세요
          # Separate variables for each function
          self.D_vars = tf.get_collection() #D에 관한 변수를 모을 collection
          self.G_vars = tf.get_collection() #G에 관한 변수를 모을 collection


          # generating images for sample
          self.sample_data = self.Generator(self.sample_random_z, is_training=False, reuse=True)

          # write summaries
          # Add loss summaries
          tf.summary.scalar('losses/loss_Discriminator', self.loss_Discriminator)
          tf.summary.scalar('losses/loss_Generator', self.loss_Generator)

          # Add histogram summaries
          for var in self.D_vars:
            tf.summary.histogram(var.op.name, var)
          for var in self.G_vars:
            tf.summary.histogram(var.op.name, var)

          # Add image summaries
          tf.summary.image('random_images', self.generated_data, max_outputs=4)
          tf.summary.image('real_images', self.real_data)

    print('complete model build.\n')

In [51]:
opt_D = tf.train.AdamOptimizer(learning_rate=learning_rate_D, beta1=0.5)
opt_G = tf.train.AdamOptimizer(learning_rate=learning_rate_G, beta1=0.5)

In [52]:
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope='Discriminator')):
  opt_D_op = opt_D.minimize(model.loss_Discriminator, var_list=model.D_vars)
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS, scope='Generator')):
  opt_G_op = opt_G.minimize(model.loss_Generator, global_step=model.global_step,
                            var_list=model.G_vars)

In [ ]:
### sample gan

In [ ]:
class DCGAN(object):
    
    def __init__(self, mode, train_dataset, test_dataset = None):
        """Basic setup.

        Args:
          mode (`string`): "train" or "generate".
          train_dataset (`tf.data.Dataset`): train_dataset.
          test_dataset (`tf.data.Dataset`): test_dataset.
        """
        assert mode in ["train", "generate"]
        self.mode = mode
        self.x_dim = 28 #mnist image dim
        self.z_dim = 100 #Random Noise dim
        self.batch_size = batch_size
        self.num_samples = num_samples
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        
        #Global step Tensor
        self.global_step = None
        
        print('The mode is %s.' % self.mode)
        print('complete initializing model.')
        
    def build_random_z_inputs(self, dataset):
        """Build a vector random_z in latent space.

        Returns:
          self.random_z (`2-rank Tensor` with [batch_size, z_dim]):
              latent vector which size is generally 100 dim.
          self.sample_random_z (`2-rank Tensor` with [num_samples, z_dim]):
              latent vector which size is generally 100 dim.
        """
        with tf.variable_op_scope('Random_z'):
            self.random_z = tf.random_uniform(shape=[self.batch_size, z_dim], minval= -1.0, maxval= 1.0)
            self.sample_random_z = tf.random_uniform(shape=[self.num_samples, z_dim], minval=-1.0, maxval= 1.0)
            
        return self.random_z, self.sample_random_z
         
        
    def read_MNIST(self, dataset):
        """Read MNIST dataset

        Args:
          dataset (`tf.data.Dataset` format): MNIST dataset.

        Returns:
          self.mnist (`4-rank Tensor` with [batch, x_dim, x_dim, 1]): MNIST dataset with batch size.
        """
        with tf.variable_op_scope('Read_mnist'):
            iterator = dataset.make_one_shot_iterator

            self.mnist = iterator.get_next()
            self.mnist = tf.cast(self.mnist, dtype=tf.float32)
            self.mnist = tf.expand_dims(self.mnist, axis=3)
        
        return self.mnist
    
    def Generator(self, random_z, is_training = True, reuse = False):
        """Generator setup.

        Args:
          random_z (`2-rank Tensor` with [batch_size, z_dim]):
              latent vector which size is generally 100 dim.
          is_training (`bool`): whether training mode or test mode.
          reuse (`bool`): whether variable reuse or not.

        Returns:
          generated_data (`4-rank Tensor` with [batch_size, h, w, c])
              generated images from random vector z.
        """
        with tf.variable_scope('Generator', reuse=reuse):
            batch_norm_params = {'decay' :0.9, 'epsilion':0.0001,
                                'is_training' : is_training, 'scope':'batch_norm'}
            with slim.arg_scope([slim.conv2d_transpose],
                               kernel_size = [4,4], stride = [2,2],
                               normalizer_fn = slim.batch_norm,
                               normalizer_params = batch_norm_params):
                #Use full conv2d_transpose intead of projection and reshape
                #random_z = [batch_size, z_dim]
                self.inputs = random_z
                #inputs = 1 x 1 x 100 dim ??
                #outputs = 3 x 3 x 256 dim
                self.layer1 = slim.conv2d_transpose(inputs=self.inputs, num_outputs=256, padding= 'VALID', scope='layer1'
                                                   pa)
                
            
        